<a href="https://colab.research.google.com/github/maguid28/EE6041-TAandNLP/blob/main/Etivity5_2_23222425.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Task 1

In [ ]:
import numpy as np
from collections import Counter

#Training data

In [ ]:
megadoc_GB_IE_train = [
    ("London is the Capital of GB", 1),
    ("Oxford is a city in GB", 1),
    ("Dublin is the capital of Ireland", 0),
    ("Limerick is a city in Ireland", 0),
]

megadoc_IE_train = [
    ("Dublin is the capital of Ireland", 0),
    ("Limerick is a city in Ireland", 0),
]

megadoc_GB_train = [
    ("Oxford is a city in GB", 1),
    ("London is the Capital of GB", 1),
]

#Test Dataset

In [ ]:
megadoc_test_dataset = [
    ("University of Limerick", "?"),
    ("University College Dublin", "?"),
    ("Imperial College London", "?"),
    ("University of Oxford", "?"),
    ("Ireland & GB", "?"),
]

#Priors

In [ ]:
def calculate_prior_probabilities(dataset):
    total_samples = len(dataset)
    print(f"total samples:  {total_samples}")
    GB_samples = sum(label for _, label in dataset)
    IE_samples = total_samples - GB_samples

    print(dataset)

    prior_probability_GB = GB_samples / total_samples
    prior_probability_IE = IE_samples / total_samples

    return prior_probability_GB, prior_probability_IE

In [ ]:
priorIE, priorGB = calculate_prior_probabilities(megadoc_GB_IE_train)
priors = calculate_prior_probabilities(megadoc_GB_IE_train)

total samples:  4
[('London is the Capital of GB', 1), ('Oxford is a city in GB', 1), ('Dublin is the capital of Ireland', 0), ('Limerick is a city in Ireland', 0)]
total samples:  4
[('London is the Capital of GB', 1), ('Oxford is a city in GB', 1), ('Dublin is the capital of Ireland', 0), ('Limerick is a city in Ireland', 0)]


#Bag of Words

In [ ]:
def create_bow(dataset, min_frequency = 1):
    all_words = []
    for sample, _ in dataset:
        all_words.extend(sample.lower().split())

    word_frequencies = {}
    for word in all_words:
        if word in word_frequencies:
            word_frequencies[word] += 1
        else:
            word_frequencies[word] = 1

    bag_of_words = {word: count for word, count in word_frequencies.items() if count >= min_frequency}
    return bag_of_words

In [ ]:
bag_of_words_IE = create_bow(megadoc_IE_train)

In [ ]:
bag_of_words_GB = create_bow(megadoc_GB_train)

NameError: ignored

In [ ]:
def create_combined_bow(*datasets):
    combined_bow = {}
    for dataset in datasets:
        for sample, _ in dataset:
            words = sample.lower().split()
            for word in words:
                combined_bow[word] = combined_bow.get(word, 0) + 1
    return combined_bow

combined_bow = create_combined_bow(megadoc_IE_train, megadoc_GB_train)

#Likelihoods

In [ ]:
def calculate_likelihoods(dataset, bag_of_words):

    laplace_smoothing = 1
    gb_samples = [text.lower() for text, label in dataset if label == 1]
    ie_samples = [text.lower() for text, label in dataset if label == 0]

    gb_word_counts = {word: laplace_smoothing for word in bag_of_words}
    ie_word_counts = {word: laplace_smoothing for word in bag_of_words}

    for text in gb_samples:
        for word in text.split():
            if word in gb_word_counts:
                gb_word_counts[word] += 1

    for text in ie_samples:
        for word in text.split():
            if word in ie_word_counts:
                ie_word_counts[word] += 1

    gb_total_count = sum(gb_word_counts.values())
    ie_total_count = sum(ie_word_counts.values())

    gb_likelihoods = {word: gb_word_counts[word] / gb_total_count for word in gb_word_counts}
    ie_likelihoods = {word: ie_word_counts[word] / ie_total_count for word in ie_word_counts}

    return gb_likelihoods, ie_likelihoods

#Posterior Prob

In [ ]:
def posterior_probability(new_sample, prior_probabilities, likelihoods):
    ie_likelihoods, gb_likelihoods = likelihoods
    prior_probability_ie, prior_probability_gb = prior_probabilities

    words = new_sample.lower().split()

    log_gb_prob = np.log(prior_probability_gb)
    log_ie_prob = np.log(prior_probability_ie)

    for word in words:
        gb_likelihood = gb_likelihoods.get(word, 1 / (sum(gb_likelihoods.values()) + len(gb_likelihoods)))
        ie_likelihood = ie_likelihoods.get(word, 1 / (sum(ie_likelihoods.values()) + len(ie_likelihoods)))

        print(f"Word: {word} \t wordConditionalProbGB: {gb_likelihood} \t wordConditionalProbIE: {ie_likelihood}")

        log_gb_prob += np.log(gb_likelihood)
        log_ie_prob += np.log(ie_likelihood)

    #Convert back to normal probabilities
    gb_prob = np.exp(log_gb_prob)
    ie_prob = np.exp(log_ie_prob)

    total_prob = gb_prob + ie_prob
    posterior_probability_gb = gb_prob / total_prob
    posterior_probability_ie = ie_prob / total_prob

    return posterior_probability_gb, posterior_probability_ie


In [ ]:
GB_likelihoods, IE_likelihoods = calculate_likelihoods(megadoc_GB_IE_train, combined_bow)
likelihoods = (IE_likelihoods, GB_likelihoods)

NameError: ignored

#Classify

In [ ]:
def classify_text(sentence, prior_probabilities, likelihoods):
    posterior_probability_gb, posterior_probability_ie = posterior_probability(sentence, prior_probabilities, likelihoods)
    print(f"\ndocProbGB: {posterior_probability_gb}")
    print(f"docProbIE: {posterior_probability_ie}")

    if posterior_probability_gb > posterior_probability_ie:
        classification = "GB"
    elif posterior_probability_ie > posterior_probability_gb:
        classification = "IE"
    else:
        classification = "Neutral"

    return classification

#IE Likelihoods

In [ ]:
IE_likelihoods

{'dublin': 0.08,
 'is': 0.12,
 'the': 0.08,
 'capital': 0.08,
 'of': 0.08,
 'ireland': 0.12,
 'limerick': 0.08,
 'a': 0.08,
 'city': 0.08,
 'in': 0.08,
 'oxford': 0.04,
 'gb': 0.04,
 'london': 0.04}

#GB Likelihoods

In [ ]:
GB_likelihoods

{'dublin': 0.04,
 'is': 0.12,
 'the': 0.08,
 'capital': 0.08,
 'of': 0.08,
 'ireland': 0.04,
 'limerick': 0.04,
 'a': 0.08,
 'city': 0.08,
 'in': 0.08,
 'oxford': 0.08,
 'gb': 0.12,
 'london': 0.08}

#Classify Test Docs

In [ ]:
print(f"megadocIE: {megadoc_IE_train}")
print(f"megadocGB: {megadoc_GB_train}\n")

print(f"IE_BOW = {bag_of_words_IE}")
print(f"gb_BOW = {bag_of_words_GB}\n")

print(f"probIE: {priorIE}")
print(f"probGB: {priorGB}\n")

vocab_size = 0
for i in combined_bow :
  vocab_size+=1
print(f"V = {combined_bow}")
print(f"|V| = {vocab_size}\n")

for doc in megadoc_test_dataset :
  print("--------------------------------------------------------------------------------")
  print(f"\nTest Document: {doc} \n")
  classified_result = classify_text(doc[0], priors, likelihoods)
  print(f"\nInferred Class: {classified_result}")

megadocIE: [('Dublin is the capital of Ireland', 0), ('Limerick is a city in Ireland', 0)]
megadocGB: [('Oxford is a city in GB', 1), ('London is the Capital of GB', 1)]

IE_BOW = {'dublin': 1, 'is': 2, 'the': 1, 'capital': 1, 'of': 1, 'ireland': 2, 'limerick': 1, 'a': 1, 'city': 1, 'in': 1}
gb_BOW = {'oxford': 1, 'is': 2, 'a': 1, 'city': 1, 'in': 1, 'gb': 2, 'london': 1, 'the': 1, 'capital': 1, 'of': 1}

probIE: 0.5
probGB: 0.5

V = {'dublin': 1, 'is': 4, 'the': 2, 'capital': 2, 'of': 2, 'ireland': 2, 'limerick': 1, 'a': 2, 'city': 2, 'in': 2, 'oxford': 1, 'gb': 2, 'london': 1}
|V| = 13

--------------------------------------------------------------------------------

Test Document: ('University of Limerick', '?') 

Word: university 	 wordConditionalProbGB: 0.07142857142857142 	 wordConditionalProbIE: 0.07142857142857142
Word: of 	 wordConditionalProbGB: 0.08 	 wordConditionalProbIE: 0.08
Word: limerick 	 wordConditionalProbGB: 0.04 	 wordConditionalProbIE: 0.08

docProbGB: 0.33333333